<a href="https://colab.research.google.com/github/Jxelr/RNA/blob/main/Series_de_Tiempo_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras import utils

In [2]:
"""Descarga y descomprime los datos"""
!wget https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
!unzip jena_climate_2009_2016.csv.zip

--2023-10-27 07:12:21--  https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 16.182.101.16, 54.231.130.24, 52.217.128.208, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|16.182.101.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13565642 (13M) [application/zip]
Saving to: ‘jena_climate_2009_2016.csv.zip’

jena_climate_2009_2 100%[===================>]  12.94M  47.2MB/s    in 0.3s    

2023-10-27 07:12:21 (47.2 MB/s) - ‘jena_climate_2009_2016.csv.zip’ saved [13565642/13565642]

Archive:  jena_climate_2009_2016.csv.zip
  inflating: jena_climate_2009_2016.csv  
  inflating: __MACOSX/._jena_climate_2009_2016.csv  


In [5]:
"""Imprimimos los encabezados y separamos los datos del encabezado"""

fname = os.path.join("jena_climate_2009_2016.csv")
with open(fname) as f:
   data = f.read()

lines = data.split("\n")
header = lines[0].split(",")
lines = lines[1:]
print(header)
print(len(lines))

['"Date Time"', '"p (mbar)"', '"T (degC)"', '"Tpot (K)"', '"Tdew (degC)"', '"rh (%)"', '"VPmax (mbar)"', '"VPact (mbar)"', '"VPdef (mbar)"', '"sh (g/kg)"', '"H2OC (mmol/mol)"', '"rho (g/m**3)"', '"wv (m/s)"', '"max. wv (m/s)"', '"wd (deg)"']
420451


In [6]:
"""Separamos Datos"""
temperature = np.zeros((len(lines),))
raw_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
  values = [float(x) for x in line.split(",")[1:]]
  temperature[i] = values[1]
  raw_data[i, :] = values[:]

In [10]:
"""Definimos la longitud de la secuencia y el paso de la prediccción"""
sequence_length = 120
prediction_step = 10 #10 pasos al futuro

In [11]:
# Dividimos los datos en secuencias de entrada y etiquetas de salida

input_sequences = []
output_labels = []

for i in range(len(temperature) - sequence_length - prediction_step):
  input_sequences.append(temperature[1:1 + sequence_length])
  output_labels.append(temperature[i + sequence_length + prediction_step])


input_sequences = np.array(input_sequences)
output_labels = np.array(output_labels)

In [12]:
#Dividimos los datos en coinjuntos de entrenamiento y prueba (train. test)

train_size = int(0.7 * len(input_sequences)) #70% de los datos van al entrenamiento
train_input = input_sequences[:train_size]
train_output = output_labels[:train_size]
test_input = input_sequences[train_size:]
test_output = output_labels[train_size:]

In [14]:
#Construimos un modelo LSTM

model = keras.Sequential()
model.add(layers.LSTM(50, input_shape=(sequence_length, 1)))
model.add(layers.Dense(1))

In [15]:

# Compila el modelo
model.compile(optimizer='adam', loss='mse')

#Entrena el Modelo
model.fit(train_input, train_output, epochs=10, batch_size=20, validation_data=(test_input, test_output))


Epoch 1/10
14712/14712 [==============================] - 493s 33ms/step - loss: 75.0016 - val_loss: 61.9692
Epoch 2/10
14712/14712 [==============================] - 450s 31ms/step - loss: 74.8615 - val_loss: 62.5965
Epoch 3/10
14712/14712 [==============================] - 449s 30ms/step - loss: 74.8579 - val_loss: 62.8424
Epoch 4/10
14712/14712 [==============================] - 447s 30ms/step - loss: 74.8574 - val_loss: 62.1907
Epoch 5/10
14712/14712 [==============================] - 448s 30ms/step - loss: 74.8615 - val_loss: 62.7633
Epoch 6/10
14712/14712 [==============================] - 448s 30ms/step - loss: 74.8552 - val_loss: 61.7314
Epoch 7/10
14712/14712 [==============================] - 447s 30ms/step - loss: 74.8576 - val_loss: 62.7057
Epoch 8/10
14712/14712 [==============================] - 448s 30ms/step - loss: 74.8551 - val_loss: 62.3780
Epoch 9/10
14712/14712 [==============================] - 451s 31ms/step - loss: 74.8562 - val_loss: 62.2072
Epoch 10/10
14712/1

In [16]:
#Evalúa el modelo
loss = model.evaluate(test_input, test_output)
print(f'pérdida en el conjunto de prueba: {loss}')

3941/3941 [==============================] - 33s 8ms/step - loss: 62.1068
pérdida en el conjunto de prueba: 62.1068000793457


In [17]:
#Se hace una predicción de temperatura 10 pasos en el futuro
input_sequence = temperature[-sequence_length:]
input_sequence = np.reshape(input_sequence, (1, sequence_length, 1))
predicted_temperature = model.predict(input_sequence)
print(f'Temperatura predicha 10 pasos en el futuro: {predicted_temperature[0][0]}')

1/1 [==============================] - 0s 307ms/step
Temperatura predicha 10 pasos en el futuro: 9.124300956726074
